In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append("../")

location = "remote"
if location == "remote":
    # TODO: hacky, shouldn't be necessary
    os.environ["WANDB_NOTEBOOK_NAME"] = "lustre_scratch/coralshift/notebooks/convlstm.ipynb"
    os.chdir("/lustre_scratch/orlando-code/coralshift/")


In [2]:
from __future__ import annotations

from pathlib import Path
import xarray as xa
import numpy as np

import wandb
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
# importing entire keras library leads to quasi-errors: https://stackoverflow.com/questions/65493824/tensorflow-gpu-memory-allocation
from tensorflow.keras import layers, models, losses, optimizers, callbacks


import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbCallback


# from sklearn import model_selection
# from sklearn.preprocessing import normalize
# from scipy.interpolate import interp2d
# from sklearn.utils import class_weight

import xbatcher

from sklearn.utils import class_weight
# import rasterio
# # from rasterio.plot import show
# import rioxarray as rio

# # from bs4 import BeautifulSoup
# # import requests


#issues with numpy deprecation in pytorch_env
from coralshift.processing import spatial_data
from coralshift.utils import file_ops, directories
from coralshift.plotting import spatial_plots, model_results
from coralshift.dataloading import data_structure, climate_data

from keras.models import Sequential
from keras.layers import Conv3D
from keras.layers import ConvLSTM2D
from keras.layers import BatchNormalization
# import keras
import numpy as np
import pylab as plt

from time import sleep
from tqdm.notebook import tqdm

# from keras.layers import Flatten

2023-06-17 10:23:49.118930: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
rows, cols = 40, 60
seq_len = 10
num_fs = 5
target_frac = 0.1
num_samples = 32

# Creating the features array in form (sequence_length, width, height, num_features)
features = np.random.random((num_samples, seq_len, rows, cols, num_fs)).astype(np.float32)

# Creating the label array
labels = np.repeat(np.expand_dims(
    np.random.choice((2), size=(rows, cols), p=[1-target_frac,target_frac]).reshape((1,40,60,1)), axis=0),
    num_samples, axis=0)

batches_X = np.repeat(np.expand_dims(features,axis=0), 100, axis=0)

class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels.flatten())

# Verifying the shape of the arrays
print("Features shape:", features.shape)
print("Label shape:", labels.shape)
print("Class weights:", class_weights)


# send numpy features/label array to tf.Data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((features, labels))

BATCH_SIZE = 8
SHUFFLE_BUFFER_SIZE = 10

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
print("Number of batches:", len(train_dataset))

Features shape: (32, 10, 40, 60, 5)
Label shape: (32, 1, 40, 60, 1)
Class weights: [0.55223194 5.28634361]


2023-06-17 10:24:04.768261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Number of batches: 4


2023-06-17 10:24:05.871687: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31017 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:06:00.0, compute capability: 7.0
2023-06-17 10:24:05.873957: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 31017 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:8a:00.0, compute capability: 7.0


In [ ]:
def negative_log_likelihood(y: np.ndarray, y_pred: np.ndarray, class_weights: np.ndarray = None) -> float:
    """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
    incorporating class weights.

    Parameters
    ----------
    y (np.ndarray): true binary labels, where 0 represents the negative class
    y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)
    class_weights (np.ndarray): weights for each class. If None, no class weights will be applied.

    Returns
    -------
    float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred',
    incorporating class weights if provided
    """
    bce = tf.keras.losses.BinaryCrossentropy()  

    if class_weights is not None:
        sample_weights = tf.gather(class_weights, np.asarray(y,dtype=np.int32))
        return bce(y, y_pred, sample_weight=tf.reshape(sample_weights, (-1, 1)))

    return bce(y, y_pred)

In [ ]:
# features.shape[2:]
inp = layers.Input(shape=(None, *features.shape[1:]))
inp.shape

In [ ]:
# # Construct the input layer with no definite frame size.
# inp = layers.Input(shape=(features.shape[1:]))
# # inp = layers.Input(shape=(None, *x_train.shape[2:]))

# # TODO: add config file, potentially with smaller learning rate
# # https://stackoverflow.com/questions/40050397/deep-learning-nan-loss-reasons
# # mirrored_strategy leads to nan loss values
# mirrored_strategy = tf.distribute.MirroredStrategy()

# with mirrored_strategy.scope():

#     # We will construct 3 `ConvLSTM2D` layers with batch normalization
#     # 
#     x = layers.ConvLSTM2D(
#         filters=64,
#         kernel_size=(5, 5),
#         padding="same",
#         return_sequences=True,
#         activation="relu",
#     )(inp)
#     x = layers.BatchNormalization()(x)
#     x = layers.ConvLSTM2D(
#         filters=64,
#         kernel_size=(3, 3),
#         padding="same",
#         return_sequences=True,
#         activation="relu",
#     )(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ConvLSTM2D(
#         filters=64,
#         kernel_size=(1, 1),
#         padding="same",
#         return_sequences=True,
#         activation="relu",
#     )(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.ConvLSTM2D(
#         filters=64,
#         kernel_size=(1, 1),
#         padding="same",
#         return_sequences=True,
#         activation="relu",
#     )(x)

#     # Reshape the output tensor to match the target shape (num_samples, 1, rows, cols, 1)
#     # x = layers.Reshape((1, *x.shape[2:]))(x)
#     # x= layers.TimeDistributed(layers.Dense(1))(x)
#     x= layers.Dense(1)(x)

#     # # Add a final Conv3D layer for the spatiotemporal outputs
#     # x = layers.Conv3D(
#     #     filters=8, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
#     # )(x)

#     # Next, we will build the complete model and compile it.
#     model = models.Model(inputs=inp,outputs=x)

#     # TODO: include weighting of classes: https://stackoverflow.com/questions/46009619/keras-weighted-binary-crossentropy
#     model.compile(
#         loss=losses.binary_crossentropy,
#         optimizer=optimizers.Adam(learning_rate=1e-3),
#     )

# model.summary()

In [ ]:
print(np.shape(features))
print(np.shape(labels))

In [ ]:
features.shape

In [ ]:
labels = labels.reshape((1,40,60,1))

In [ ]:
plt.imshow(labels[0,:,:,0])

In [ ]:
batches_X = np.repeat(np.expand_dims(features,axis=1), 100, axis=0)
batches_y = np.repeat(np.expand_dims(labels,axis=1), 100, axis=0)

print("batches_X:", batches_X.shape)
print("batches_y:", batches_y.shape)

In [ ]:
duplicated_batches_X = np.tile(batches_X, (1, batches_y.shape[1], 1, 1, 1))
duplicated_batches_X.shape
duplicated_batches_y = np.tile(batches_y, (1, batches_X.shape[1], 1, 1, 1))
duplicated_batches_y.shape

In [ ]:
# # Define some callbacks to improve training. Patience measured in number of epochs.
# early_stopping = callbacks.EarlyStopping(monitor="val_loss", patience=10)
# reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# # Define modifiable training hyperparameters.
# epochs = 4
# batch_size = 32

# # Start a run, tracking hyperparameters
# # wandb.init(
# #     # set the wandb project where this run will be logged
# #     project="coralshift_test",
# # )


# # with tf.device("/GPU:0"):
# # Fit the model to the training data.
# model.fit(
#     # x_train,
#     # y_train,
#     batches_X,
#     batches_y,
#     # class_weight=class_weights,
#     # batch_size=batch_size,
#     epochs=epochs,
#     # validation_data=(x_val, y_val),
#     validation_data=(batches_X, batches_y),
#     callbacks=[early_stopping, reduce_lr,
#         # WandbMetricsLogger(log_freq=5), WandbModelCheckpoint("models"),
#         # WandbCallback(training_data=(padded_spatial_Xs,padded_spatial_ys), log_weights=True, log_gradients=True)
# ])

# # wandb.finish()

In [ ]:
# tf.config.run_functions_eagerly(False)

In [ ]:
# X_train = features
# y_train = labels

# # wandb.init(
# #     project="coralshift",
# #     entity="orlando-code",
# #     settings=wandb.Settings(start_method="fork")
# #     # config={    }
# #     )

# # initialize optimiser: will need hyperparameter scan for learning rate and others
# # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
# optimizer = tf.keras.optimizers.Adam(3e-4)

# # X = ds_man.get_dataset("monthly_climate_1_12_X_np")
# # y = ds_man.get_dataset("monthly_climate_1_12_y_np")
# # # check that untrained model runs (should output array of non-nan values)
# # # why values change?
# # # g_model(X[:32])

# # X_train, X_test, y_train, y_test = model_selection.train_test_split(
# #     X, y, test_size=0.2, random_state=42)

# # X_train, X_test, y_train, y_test = model_selection.train_test_split(
# #     sub_X, sub_y, test_size=0.2, random_state=42)



# # Define Convolution LSTM Model
# class ConvLSTModel(tf.keras.Model):
#     # initialise class instance to define layers of the model
#     def __init__(self, filters: list[int] = [32, 64, 128], kernel_sizes: list[int] = [3,3,3], strides = list[int] = [1,1,1],
#         input_shape: tuple[int] = INPUT_SHAPE, num_layers: int = 3):

#         # initialise CoonvLSTModel model as subclass of tf.keras.Model
#         super(ConvLSTModel, self).__init__()

#         # check argument consistency
#         assert(len(filters) == num_layers)
#         assert(len(kernel_sizes) == len(filters))
#         assert(len(strides) == len(filters))

#         # TODO: put outside in wandb config
#         activation = "relu"

#         self.conv_lstm_0 = layers.ConvLSTM2D(filters=filters[0],
#                                             kernel_size=(kernel_sizes[0], kernel_sizes[0]),
#                                             strides=(strides[0], strides[0])
#                                             padding=padding,
#                                             return_sequences=True,
#                                             activation=activation)


#         self.conv_lstm_1 = layers.ConvLSTM2D(filters=filters[1],
#                                             kernel_size=(kernel_sizes[1], kernel_sizes[1]),
#                                             strides=(strides[1], strides[1])
#                                             padding=padding,
#                                             return_sequences=True,
#                                             activation=activation)


#         self.conv_lstm_2 = layers.ConvLSTM2D(filters=filters[2],
#                                             kernel_size=(kernel_sizes[2], kernel_sizes[2]),
#                                             strides=(strides[2], strides[2])
#                                             padding=padding,
#                                             return_sequences=True,
#                                             activation=activation)

#         self.batch_norm_0 = layers.BatchNormalization()
#         self.batch_norm_1 = layers.BatchNormalization()
#         self.batch_norm_2 = layers.BatchNormalization()

#         # reducing parameter space to 1 to match label
#         self.dense_layer_out = layers.Dense(1)


#         # mirrored_strategy = tf.distribute.MirroredStrategy()

#     def call(self, inputs, training=False)
#         # for now, threee convLSTM2D layers with batchnorm sandwiches
#         x = self.conv_lstm_0(inputs)
#         x = self.batch_norm_0(x, training=training)
#         x = self.conv_lstm_1(x)
#         x = self.batch_norm_1(x, training=training)
#         x = self.conv_lstm_2(x)
#         x = self.batch_norm_2(x, training=training)
#         x = self.dense_layer_out(x)

#         return x



#         # Reshape the output tensor to match the target shape (num_samples, 1, rows, cols, 1)
#         # x = layers.Reshape((1, *x.shape[2:]))(x)
#         x= layers.TimeDistributed(layers.Dense(1))(x)

#         # # Add a final Conv3D layer for the spatiotemporal outputs
#         # x = layers.Conv3D(
#         #     filters=8, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
#         # )(x)

#         # Next, we will build the complete model and compile it.
#         model = models.Model(inputs=inp,outputs=x)

#         # TODO: include weighting of classes: https://stackoverflow.com/questions/46009619/keras-weighted-binary-crossentropy
#         model.compile(
#             loss=losses.binary_crossentropy,
#             optimizer=optimizers.Adam(learning_rate=1e-3),
#         )

# model.summary()

#         """Sets up a GRU model architecture with multiple layers and dense layers for mapping the outputs of the GRU 
#         layers to a desired output shape

#         Parameters
#         ----------
#         rnn_units (list[int]): list containing the number of neurons to use in each layer
#         num_layers (int): number of layers in GRU model
#         """
#         super(gru_model, self).__init__()   # initialise GRU model as subclass of tf.keras.Model
#         # store values for later use
#         self.num_layers = num_layers    # number of layers in GRU model
#         self.rnn_units = rnn_units
#         # self.dff = dff
#         # define model layers: creating new `tf.keras.layers.GRU` layer for each iteration
#         self.grus = [tf.keras.layers.GRU(rnn_units[i],  # number (integer) of rnn units/neurons to use in each model layer
#                                    return_sequences=True,   # return full sequence of outputs for each timestep
#                                    return_state=True) for i in range(num_layers)] # return last hidden state of RNN at end of sequence
        
#         # dense layers are linear mappings of RNN layer outputs to desired output shape
#         # self.w1 = tf.keras.layers.Dense(dff) # 10 units
#         self.w1 = tf.keras.layers.Dense(10) # 10 units

#         self.w2 = tf.keras.layers.Dense(1)  # 1 unit (dimension 1 required before final sigmoid function)
#         # self.A = tf.keras.layers.Dense(30)
#         # self.B = tf.keras.layers.Dense(dff)



#     def call(self, inputs: np.ndarray, training: bool=False):
#         """Processes an input sequence of data through several layers of GRU cells, followed by a couple of
#         fully-connected dense layers, and outputs the probability of an event happening.
        
#         Parameters
#         ----------
#         inputs (np.ndarray): input tensor of shape (batch_size, seq_length, features)
#             batch_size - defines the size of the sample drawn from datapoints
#             seq_length - number of timesteps in sequence
#             features - number of features associated with each datapoint
#         training (bool, defaults to False): True if model is in training, False if in inference mode

#         Returns
#         -------
#         target: probability of an event occuring, with shape (batch_size, 1)
#         """
#         # input shape: (batch_size, seq_length, features)
       
#         assert self.num_layers == len(self.rnn_units)

#         # check that input tensor has correct shape
#         if (len(inputs.shape) != 3):
#             print(f"Incorrect shape of input tensor. Expected 3D array. Recieved {len(inputs.shape)}D array.")

#         # print('input dim ({}, {}, {})'.format(inputs.shape[0], inputs.shape[1], inputs.shape[2]))
#         # whole_seq, static_input = inputs
#         whole_seq = inputs


#         # iteratively passes input tensor to GRU layers, overwriting preceding sequence 'whole_seq'
#         for layer_num in range(self.num_layers):
#             whole_seq, final_s = self.grus[layer_num](whole_seq, training=training)

#         # adding extra layers
#         # static = self.B(tf.nn.gelu(self.A(static_input)))
#         # target = self.w1(final_s)  + static # final hidden state of last layer used as input to fully connected dense layers...
#         target = self.w1(final_s)   # final hidden state of last layer used as input to fully connected dense layers...

#         target = tf.nn.relu(target) # via ReLU activation function
#         target = self.w2(target)    # final hidden layer must have dimension 1 
        
#         # obtain a probability value between 0 and 1
#         target = tf.nn.sigmoid(target)
        
#         return target


# # initialise GRU model with 500 hidden layers, one GRU unit per layer 
# g_model = gru_model([100], 1) # N.B. [x] is number of hidden layers in GRU network


# def negative_log_likelihood(y: np.ndarray, y_pred: np.ndarray, class_weights: np.ndarray = None) -> float:
#     """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
#     incorporating class weights.

#     Parameters
#     ----------
#     y (np.ndarray): true binary labels, where 0 represents the negative class
#     y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)
#     class_weights (np.ndarray): weights for each class. If None, no class weights will be applied.

#     Returns
#     -------
#     float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred',
#     incorporating class weights if provided
#     """
#     bce = tf.keras.losses.BinaryCrossentropy()  

#     if class_weights is not None:
#         sample_weights = tf.gather(class_weights, np.asarray(y,dtype=np.int32))
#         # reshape to match size of y and y_pred
#         return bce(y, y_pred, sample_weight=tf.reshape(sample_weights, (-1, 1)))

#     return bce(y, y_pred)


# def training_batches(X: np.ndarray, y: np.ndarray, batch_num: int, batch_size: int=32):
#     start_idx = batch_num * batch_size
#     end_idx = (batch_num + 1) * batch_size

#     X_batch = X[start_idx:end_idx]
#     y_batch = y[start_idx:end_idx]
    
#     return X_batch, y_batch

# # https://stackoverflow.com/questions/52357542/attributeerror-tensor-object-has-no-attribute-numpy
# # should aim to delete the following to speed up training: but can't figure out a way to make wandb reporting work
# # without it
# tf.config.run_functions_eagerly(True)

# def build_graph():
    
#     # compile function as graph using tf's autograph feature: leads to faster execution times, at expense of limitations
#     # to Python objects/certain control flow structures (somewhat relaxed by experimental_relax_shapes)
#     @tf.function(experimental_relax_shapes=True)
#     def train_step(gru: tf.keras.Model, optimizer: tf.keras.optimizers.Optimizer, X: tf.Tensor, y: tf.Tensor, 
#                training: bool = True, class_weights=class_weights) -> tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
#         """Train model using input `X` and target data `y` by computing gradients of the loss (via 
#         negative_log_likelihood)
        
#         Parameters
#         ----------
#         y (np.ndarray): true binary labels, where 0 represents the negative class
#         y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)

#         Returns
#         -------
#         float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred'
#         """
#         if training:
#             # num_samples = X.shape[0]
#             # num_batches = num_samples // batch_size
#             # num_batches = batch_num
#             # total_epoch_loss = 0.0
#             # for batch_num in tqdm(range(num_batches), desc="batches", position=0, leave=True):
#             # for batch_num, batch in tqdm(enumerate(ds), desc="batches", position=0, leave=True):
#             total_batch_loss = 0

#             y_pred = gru(X, training)
#             xent = negative_log_likelihood(y, y_pred, class_weights)

#                 # X_batch, y_batch = training_batches(X, y, batch_num=batch_num, batch_size=batch_size)

#             with tf.GradientTape(persistent=True) as tape:
#                 y_pred = gru(X, training) 
#                 xent = negative_log_likelihood(y, y_pred, class_weights)
#                 # y_pred = gru(X, training) # TO DELETE
#                 # xent = negative_log_likelihood(y, y_pred)
            
#             gradients = tape.gradient(xent, gru.trainable_variables)
#             optimizer.apply_gradients(zip(gradients, gru.trainable_variables))
#             # print("xent", xent.numpy())
#             # print("total_epoch_loss", total_epoch_loss)
#             total_batch_loss += xent
#                 # learning rate?
#                 # wandb.log({"batch": batch_num, "loss": xent, "total_epoch_loss": total_epoch_loss})

#             average_loss = total_batch_loss / num_batches
#             # return predicted output values and total loss value
#             return y_pred, xent, total_batch_loss

#     # set default float type
#     tf.keras.backend.set_floatx('float32')
#     # TODO: this isn't assigned... What should it return otherwise?
#     return train_step


# with tf.device("/GPU:0"):
#     num_epochs = 2
#     # will update so that subsamples are fed in from which batches are taken: will require recomputation
#     # of class_weight for each subsample

#     tr_step = build_graph()

#     for epoch in tqdm(range(num_epochs), desc= " epochs", position=1, leave=True):
#         total_epoch_loss = 0.0
#         for X_batch, y_batch in tqdm(train_dataset, position=0, desc=" training on batches", leave=True):
#             y_pred, xent, batch_loss = tr_step(
#                 g_model, optimizer, X_batch, y_batch, class_weights=class_weights, training=True)

#             total_epoch_loss += batch_loss

#         average_loss = total_epoch_loss / len(train_dataset)


# # wandb.finish()


In [ ]:




# # Define Convolution LSTM Model
# class ConvLSTModel(tf.keras.Model):
#     # initialise class instance to define layers of the model
#     def __init__(self, filters: list[int] = [32, 64, 128], kernel_sizes: list[int] = [3,3,3], 
#         strides: list[int] = [1,1,1], num_layers: int = 3, padding: str = "same",
#         # input_shape: tuple[int] = INPUT_SHAPE,
#         ):

#         # initialise CoonvLSTModel model as subclass of tf.keras.Model
#         super(ConvLSTModel, self).__init__()

#         # check argument consistency
#         assert(len(filters) == num_layers)
#         assert(len(kernel_sizes) == len(filters))
#         assert(len(strides) == len(filters))

#         # TODO: put outside in wandb config
#         activation = "relu"

#         self.conv_lstm_0 = layers.ConvLSTM2D(filters=filters[0],
#                                             kernel_size=(kernel_sizes[0], kernel_sizes[0]),
#                                             strides=(strides[0], strides[0]),
#                                             padding=padding,
#                                             return_sequences=True,
#                                             activation=activation)


#         self.conv_lstm_1 = layers.ConvLSTM2D(filters=filters[1],
#                                             kernel_size=(kernel_sizes[1], kernel_sizes[1]),
#                                             strides=(strides[1], strides[1]),
#                                             padding=padding,
#                                             return_sequences=True,
#                                             activation=activation)


#         self.conv_lstm_2 = layers.ConvLSTM2D(filters=filters[2],
#                                             kernel_size=(kernel_sizes[2], kernel_sizes[2]),
#                                             strides=(strides[2], strides[2]),
#                                             padding=padding,
#                                             return_sequences=True,
#                                             activation=activation)

#         self.batch_norm_0 = layers.BatchNormalization()
#         self.batch_norm_1 = layers.BatchNormalization()
#         self.batch_norm_2 = layers.BatchNormalization()

#         # reducing parameter space to 1 to match label
#         self.dense_layer_out = layers.Dense(5)


#     def call(self, inputs, training=False):
#         # print(inputs.shape)
#         # for now, threee convLSTM2D layers with batchnorm sandwiches
#         x = self.conv_lstm_0(inputs)
#         # print("conv0", x.shape)
#         x = self.batch_norm_0(x, training=training)
#         x = self.conv_lstm_1(x)
#         # print("conv1", x.shape)
#         x = self.batch_norm_1(x, training=training)
#         x = self.conv_lstm_2(x)
#         # print("conv2", x.shape)
#         x = self.batch_norm_2(x, training=training)
#         x = self.dense_layer_out(x)
#         # print("dense", x.shape)

#         return x


# def build_graph():
#     @tf.function
#     def train_step(conv_lstm: tf.keras.Model, features: tf.Tensor, labels: tf.Tensor, 
#         optimizer: tf.keras.optimizers.Optimizer = OPTIMIZER, class_weights: tuple = None):
#         with tf.GradientTape() as lstm_tape:

#             prediction = conv_lstm(features, training=True)
#             # Calculate loss
#             loss = negative_log_likelihood(labels, prediction, class_weights)
#             # get trainable variables
#             trainable_vars = conv_lstm.trainable_variables

#         # Calculate gradient          `
#         lstm_grads = lstm_tape.gradient(loss, trainable_vars)
#         # And then apply the gradient to change the weights
#         optimizer.apply_gradients(zip(lstm_grads, trainable_vars))

#         return loss
    
#     return train_step


# def negative_log_likelihood(y: np.ndarray, y_pred: np.ndarray, class_weights: np.ndarray = None) -> float:
#     """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
#     incorporating class weights.

#     Parameters
#     ----------
#     y (np.ndarray): true binary labels, where 0 represents the negative class
#     y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)
#     class_weights (np.ndarray): weights for each class. If None, no class weights will be applied.

#     Returns
#     -------
#     float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred',
#     incorporating class weights if provided
#     """
#     bce = tf.keras.losses.BinaryCrossentropy()  

#     if class_weights is not None:
#         sample_weights = tf.gather(class_weights, np.asarray(y,dtype=np.int32))
#         # reshape to match size of y and y_pred
#         return bce(y, y_pred, sample_weight=tf.reshape(sample_weights, (-1, 1)))

#     return bce(y, y_pred)


# conv_lstm = ConvLSTModel()

# tr_step = build_graph()


# # strategy = tf.distribute.MirroredStrategy()

# # with strategy.scope():

# for epoch in tqdm(range(EPOCHS), desc="epochs", position=0):
#     # Just total loss
#     loss_tot = 0
#     # Iterate through all the examples
#     for features, labels in tqdm(train_dataset, desc=" batches", position=1):
#         # Break if the number of computed batches exceeds the
#         # total number of the examples

#         # HERE WE PERFORM ONE TRAINING STEP
#         loss = tr_step(conv_lstm, features, labels)
#         loss_tot += loss

#     # Print the total loss every epoch
#     # print(f'\nLoss for epoch {epoch+1} is {loss_tot}')


# # # Create an instance of the model
# # model = ConvLSTModel()

# # # Print the model summary
# # model.build(shape=(None, 10, 40, 60, 5))  # Provide an example input shape
# # model.summary()

In [32]:
# class_weights
tf.gather(class_weights, np.asarray(tf.concat(labels.values, axis=0), dtype=np.int32))
# tf.concat(labels.values, axis=0)

<tf.Tensor: shape=(8, 1, 40, 60, 1), dtype=float64, numpy=
array([[[[[5.28634361],
          [0.55223194],
          [0.55223194],
          ...,
          [0.55223194],
          [0.55223194],
          [0.55223194]],

         [[0.55223194],
          [0.55223194],
          [0.55223194],
          ...,
          [5.28634361],
          [0.55223194],
          [0.55223194]],

         [[0.55223194],
          [0.55223194],
          [0.55223194],
          ...,
          [0.55223194],
          [0.55223194],
          [0.55223194]],

         ...,

         [[5.28634361],
          [0.55223194],
          [0.55223194],
          ...,
          [0.55223194],
          [0.55223194],
          [0.55223194]],

         [[0.55223194],
          [0.55223194],
          [0.55223194],
          ...,
          [0.55223194],
          [0.55223194],
          [0.55223194]],

         [[0.55223194],
          [0.55223194],
          [0.55223194],
          ...,
          [0.55223194],
          

In [41]:
np.asarray(tf.concat(labels.values, axis=0), dtype=np.int32)

array([[[[[1],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [1],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         ...,

         [[1],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]]]],



       [[[[1],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [1],
          [0],
          [0]],

         [[0],
          [0],
          [0],
          ...,
          [0],
          [0],
          [0]],

         ...,

   

In [34]:
tf.gather(class_weights, np.asarray(tf.concat(labels.values, axis=0), dtype=np.int32))


<tf.Tensor: shape=(8, 1, 40, 60, 1), dtype=float64, numpy=
array([[[[[5.28634361],
          [0.55223194],
          [0.55223194],
          ...,
          [0.55223194],
          [0.55223194],
          [0.55223194]],

         [[0.55223194],
          [0.55223194],
          [0.55223194],
          ...,
          [5.28634361],
          [0.55223194],
          [0.55223194]],

         [[0.55223194],
          [0.55223194],
          [0.55223194],
          ...,
          [0.55223194],
          [0.55223194],
          [0.55223194]],

         ...,

         [[5.28634361],
          [0.55223194],
          [0.55223194],
          ...,
          [0.55223194],
          [0.55223194],
          [0.55223194]],

         [[0.55223194],
          [0.55223194],
          [0.55223194],
          ...,
          [0.55223194],
          [0.55223194],
          [0.55223194]],

         [[0.55223194],
          [0.55223194],
          [0.55223194],
          ...,
          [0.55223194],
          

In [55]:
####################################
# DEFINE CONVOLUTIONAL LSTM MODEL
####################################


class ConvLSTModel(tf.keras.Model):
    """Convolutional LSTM model for binary classification of spatiotemporal data"""

    # initialise class instance to define layers of the model
    def __init__(self, filters: list[int] = [32, 64, 128], kernel_sizes: list[int] = [3,3,3], 
        strides: list[int] = [1,1,1], num_layers: int = 3, padding: str = "same", activation: str = "relu"
        # input_shape: tuple[int] = INPUT_SHAPE,
        ):
        """
        Initialize the ConvLSTM model (ConvLSTModel)

        Parameters
        ----------
            filters (list[int]): List of filters for each ConvLSTM2D layer.
            kernel_sizes (list[int]): List of kernel sizes for each ConvLSTM2D layer.
            strides (list[int]): List of strides for each ConvLSTM2D layer.
            num_layers (int): Number of ConvLSTM2D layers.
            padding (str): Padding mode for ConvLSTM2D layers.
            activation (str): Activation function to use.
        """

        # initialise CoonvLSTModel model as subclass of tf.keras.Model
        super(ConvLSTModel, self).__init__()

        # check argument consistency
        assert(len(filters) == num_layers)
        assert(len(kernel_sizes) == len(filters))
        assert(len(strides) == len(filters))

        self.conv_lstm_0 = layers.ConvLSTM2D(filters=filters[0],
                                            kernel_size=(kernel_sizes[0], kernel_sizes[0]),
                                            strides=(strides[0], strides[0]),
                                            padding=padding,
                                            return_sequences=True,
                                            activation=activation)


        self.conv_lstm_1 = layers.ConvLSTM2D(filters=filters[1],
                                            kernel_size=(kernel_sizes[1], kernel_sizes[1]),
                                            strides=(strides[1], strides[1]),
                                            padding=padding,
                                            return_sequences=True,
                                            activation=activation)


        self.conv_lstm_2 = layers.ConvLSTM2D(filters=filters[2],
                                            kernel_size=(kernel_sizes[2], kernel_sizes[2]),
                                            strides=(strides[2], strides[2]),
                                            padding=padding,
                                            return_sequences=True,
                                            activation=activation)

        self.batch_norm_0 = layers.BatchNormalization()
        self.batch_norm_1 = layers.BatchNormalization()
        self.batch_norm_2 = layers.BatchNormalization()

        # reducing parameter space to 1 to match label feature dimension
        self.dense_layer_out = layers.Dense(1)


    def call(self, inputs, training=False):
        # for now, threee convLSTM2D layers with batchnorm sandwiches
        x = self.conv_lstm_0(inputs)
        x = self.batch_norm_0(x, training=training)
        x = self.conv_lstm_1(x)
        x = self.batch_norm_1(x, training=training)
        x = self.conv_lstm_2(x)
        x = self.batch_norm_2(x, training=training)
        x = self.dense_layer_out(x)

        return x


###########################
# INITIALISE PARAMETERS
###########################

num_features = 5
seq_length = 100

# configure model
LEARNING_RATE = 1e-3    # TODO: adjustable (not particularly preessing though)
INPUT_SHAPE = (None, seq_length, None, None, num_features)
OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
EPOCHS = 4
ACTIVATION = "relu"
BATCH_SIZE_PER_REPLICA = 5




wandb.init(
    project="new_conv_lstm",
    entity="orlando-code",
    settings=wandb.Settings(start_method="fork")
    )

wandb.config = {
    "learning_rate": LEARNING_RATE,
    "epochs": EPOCHS,
    "activation": ACTIVATION,
    "batch_size_per_replica": BATCH_SIZE_PER_REPLICA
}


# set up distributed processing
mirrored_strategy = tf.distribute.MirroredStrategy()
GLOBAL_BATCH_SIZE = (BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync)

with mirrored_strategy.scope():
    conv_lstm = ConvLSTModel(activation=wandb.config["activation"])
    DIST_OPTIMIZER = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

    def compute_loss(
        labels: tf.Tensor, predictions: tf.Tensor, class_weights: tuple[float] = None, distributed: bool = True):
        """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
        incorporating class weights.

        Parameters
        ----------
            labels (tf.Tensor): True binary labels, where 0 represents the negative class.
            predictions (tf.Tensor): Predicted labels as probability values between 0 and 1.
            class_weights (np.ndarray): Weights for each class. If None, no class weights will be applied.
            distributed (bool): Flag indicating if the computation is distributed.

        Returns
        -------
            float: Negative log likelihood loss computed using binary cross-entropy loss between 'labels' and 
                'predictions', incorporating class weights if provided.
        """
        per_example_loss = negative_log_likelihood(labels, predictions, class_weights, distributed)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)

    train_accuracy = tf.keras.metrics.BinaryCrossentropy(name="train_accuracy")
    test_accuracy = tf.keras.metrics.BinaryCrossentropy(name="test_accuracy")
    # would use BinaryAccuracy if predicting ones/zeros

# cast datasets to mirrored versions for distributed processing
dist_train_dataset = mirrored_strategy.experimental_distribute_dataset(train_dataset)


#################################
# DEFINE TRAIN/TEST FUNCTIONS
#################################


def negative_log_likelihood(
    labels: np.ndarray, predictions: np.ndarray, class_weights: np.ndarray = None, distributed: bool=False) -> float:
    """Compute binary cross-entropy loss between ground-truth binary labels and predicted probabilities,
    incorporating class weights.

    Parameters
    ----------
    y (np.ndarray): true binary labels, where 0 represents the negative class
    y_pred (np.ndarray): predicted labels (as probability value between 0 and 1)
    class_weights (np.ndarray): weights for each class. If None, no class weights will be applied.

    Returns
    -------
    float: negative log likelihood loss computed using binary cross-entropy loss between 'y' and 'y_pred',
    incorporating class weights if provided
    """
    bce = tf.keras.losses.BinaryCrossentropy()  

    if distributed:
        bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)  

    print(tf.shape(labels))
    print(tf.shape(predictions))

    # TODO: this may be broken, throwing error associated with numpy/tensor incompatability
    if class_weights is not None:
        # sample_weights = tf.gather(class_weights, np.asarray(tf.concat(labels.values, axis=0), dtype=np.int32))
        sample_weights = tf.gather(class_weights, tf.cast(labels, dtype=tf.int32))

        # reshape to match size of y and y_pred
        return bce(labels, predictions, sample_weight=tf.reshape(sample_weights, (-1, 1)))

    return bce(labels, predictions)


@tf.function
def train_step(conv_lstm: tf.keras.Model, features: tf.Tensor, labels: tf.Tensor, 
    optimizer: tf.keras.optimizers.Optimizer = OPTIMIZER, learn_rate: float = LEARNING_RATE, 
    class_weights: tuple = None, distributed: bool=False):
    
    with tf.GradientTape() as lstm_tape:
        predictions = conv_lstm(features, training=True)
        # Calculate loss
        loss = compute_loss(labels, predictions, class_weights, distributed)
    
    # get trainable variables
    trainable_vars = conv_lstm.trainable_variables
    # Calculate gradient          `
    lstm_grads = lstm_tape.gradient(loss, trainable_vars)
    # And then apply the gradient to change the weights
    optimizer.apply_gradients(zip(lstm_grads, trainable_vars))

    train_accuracy.update_state(labels, predictions)
    return loss


def test_step(conv_lstm: tf.keras.Model, features: tf.Tensor, labels: tf.Tensor):

    predictions = conv_lstm(features, training=False)
    test_accuracy.update_state(labels, predictions)


@tf.function
def distributed_train_step(model: tf.keras.Model, features: tf.Tensor, labels: tf.Tensor, 
    optimizer: tf.keras.optimizers.Optimizer = DIST_OPTIMIZER, learn_rate: float = LEARNING_RATE,
    class_weights: tuple = None, distributed: bool = True):
    """Performs a distributed training step on the given model.

    Parameters
    ----------
        model (tf.keras.Model): The model to train.
        features (tf.Tensor): Input features for training.
        labels (tf.Tensor): True labels for training.
        optimizer (tf.keras.optimizers.Optimizer): Optimizer to use for training. Default is DIST_OPTIMIZER.
        learn_rate (float): Learning rate for the optimizer. Default is LEARNING_RATE.
        class_weights (tuple): Weights for each class. If None, no class weights will be applied. Default is None.
        distributed (bool): Flag indicating if the computation is distributed. Default is True.

    Returns
    -------
        tf.Tensor: Loss computed during the training step.
    """
    per_replica_losses = mirrored_strategy.run(train_step, args=(
        model, features, labels, optimizer, learn_rate, class_weights, distributed,))
    return mirrored_strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)


@tf.function
def distributed_test_step(conv_lstm: tf.keras.Model, features: tf.Tensor, labels: tf.Tensor):
    """Performs a distributed test step on the given model.

    Parameters
    ----------
        conv_lstm (tf.keras.Model): The model to evaluate.
        features (tf.Tensor): Input features for evaluation.
        labels (tf.Tensor): True labels for evaluation.

    Returns
    -------
        None
    """
    return mirrored_strategy.run(test_step, args=(conv_lstm, features, labels,))





################
# RUN MODEL
################

for epoch in tqdm(range(wandb.config["epochs"]), desc="epochs", position=0):
    # Just total loss
    epoch_loss = 0
    # Iterate through the train dataset
    for features, labels in tqdm(dist_train_dataset, desc=" train batches", position=1):
        loss = distributed_train_step(conv_lstm, features, labels, DIST_OPTIMIZER, wandb.config["learning_rate"],
            # class_weights
            )
        epoch_loss += loss
        wandb.log({"epoch loss": epoch_loss, "train accuracy": train_accuracy.result()})

    # Iterate through the test dataset
    for features, labels in tqdm(dist_train_dataset, desc=" test batches", position=1):
        distributed_test_step(conv_lstm, features, labels)
        wandb.log({"test accuracy": test_accuracy.result()})


wandb.finish()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2023-06-17 12:05:47.618610: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_INT64
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 32
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 10
        }
        dim {
          size: 40
        }
        dim {
          size: 60
        }
        dim {
          size: 5
        }
      }
      shape {
        dim {
          size: 1
        }
        dim {
          size: 40
        }
       

Tensor("Shape:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)
Tensor("Shape_1:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)
Tensor("replica_1/Shape:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:1)
Tensor("replica_1/Shape_1:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:1)
Tensor("Shape:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)
Tensor("Shape_1:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:0)
Tensor("replica_1/Shape:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:1)
Tensor("replica_1/Shape_1:0", shape=(5,), dtype=int32, device=/job:localhost/replica:0/task:0/device:GPU:1)


2023-06-17 12:05:58.295843: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'Func/StatefulPartitionedCall/gradient_tape/conv_lst_model_20/conv_lstm2d_62/while/conv_lst_model_20/conv_lstm2d_62/while_grad/body/_684/input/_2783' -> 'StatefulPartitionedCall/gradient_tape/conv_lst_model_20/conv_lstm2d_62/while/conv_lst_model_20/conv_lstm2d_62/while_grad/body/_684/gradient_tape/conv_lst_model_20/conv_lstm2d_62/while/gradients/AddN', 'Func/StatefulPartitionedCall/gradient_tape/replica_1/conv_lst_model_20/conv_lstm2d_62/while/replica_1/conv_lst_model_20/conv_lstm2d_62/while_grad/body/_1808/input/_3491' -> 'StatefulPartitionedCall/gradient_tape/replica_1/conv_lst_model_20/conv_lstm2d_62/while/replica_1/conv_lst_model_20/conv_lstm2d_62/while_grad/body/_1808/gradient_tape/replica_1/conv_lst_model_20/conv_lstm2d_62/while/gradients/AddN', 'Func/StatefulPartitionedCall/gradien

epoch loss,▂▄▆█▂▃▄▆▁▂▃▄▁▂▃▃
test accuracy,▁▁▁▁▃▄▅▆▆▆▅▅▆▇▇█
train accuracy,▄▅█▇▇▆▅▅▄▃▃▃▂▂▁▁
epoch loss,124629.51562
test accuracy,1.39123
train accuracy,2.6163


In [ ]:
INPUT_SHAPE = layers.Input(shape=(features.shape[1:]))
INPUT_SHAPE

In [43]:
4*10*60*40

96000

In [ ]:
inp = layers.Input(shape=(features.shape[1:]))

convlstm = ConvLSTModel()

convlstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])
# convlstm.build(INPUT_SHAPE)
# convlstm.build(features.shape[:])

convlstm.summary()

In [ ]:
inputs = tf.keras.layers.Input(shape=(3,))
outputs = tf.keras.layers.Dense(2)(inputs)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="Adam", loss="mse", metrics=["mae"])
model.metrics_names


In [ ]:
outputs

## Save model and weights

[Source](https://towardsdatascience.com/creating-a-trashy-model-from-scratch-with-tensorflow-2-an-example-with-an-anomaly-detection-27f0d1d7bd00)

In [ ]:
# encoder.save_weights(filepath='encoder_model.tf')

# # Saving the configuration
# import json
# ## Encoder
# encoder_dict = {'fillters': [32, 64, 128, 196, 256, 1024],
#                 'kernel_sizes': [3, 3, 3, 3, 3, 3],
#                 'strides': [2, 2, 2, 2, 2, 2],
#                 'num_of_layers': 6}
# with open('encoder_config.json', 'w') as fp:
#     json.dump(encoder_dict, fp)

# ## Decoder
# decoder_dict = {'fillters': [1024, 256, 196, 128, 64, 32, 1], 
#                 'kernel_sizes': [3, 3, 3, 3, 3, 3, 1],
#                 'strides': [2, 2, 2, 2, 2, 2, 1],
#                 'num_of_layers': 7}
# with open('decoder_config.json', 'w') as fp:
#     json.dump(decoder_dict, fp)
